<a href="https://colab.research.google.com/github/zhuoyanguo/BGE_Work/blob/main/LinkedinScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install selenium
# ! pip install selenium.webdriver.common.by
# ! pip install time
# ! pip install pandas
# ! pip install datetime
# ! pip install openpyxl

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import datetime

C:\Users\92767\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Step 1: Scraping information from Linkedin

In [ ]:
df = pd.read_csv('./LinkedinInput_Sample.csv')
profiles = df.iloc[:,1]

In [ ]:
# get the wait time
WAIT_TIME = 3
USERNAME = "bgecareerservices@georgetown.edu"
PASSWORD = "xxxxxxxxxx"

In [ ]:
def login(driver, username, password):
  # //div[@id='public_profile_contextual-sign-in']
  driver.get("https://www.linkedin.com/login")
  # find the username input
  username_input = driver.find_element(By.XPATH, '//input[@id="username"]')
  # send_keys() to simulate key strokes
  username_input.send_keys(username)
  # find the password input
  password_input = driver.find_element(By.XPATH, '//input[@id="password"]')
  # send_keys() to simulate key strokes
  password_input.send_keys(password)
  # find the submit button
  log_in_button = driver.find_element(By.XPATH, '//button[@type="submit"]')
  # click() to simulate button click
  log_in_button.click()
  # after login in, redirect to target profile
  time.sleep(WAIT_TIME)
  return driver

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_name(driver):
    try:
        time.sleep(3)
        name_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//h1[contains(@class, "break-words")]'))
        )
        print("Found Name:", name_element.text)
        return name_element.text.strip()

    except Exception as e:
        print(" Error getting name:", e)
        return None


In [ ]:
def get_location(driver):
  return driver.find_element(By.XPATH, '//span[contains(@class, "text-body-small inline t-black--light break-words")]').text.strip()

In [ ]:
def get_experience(driver):
  # find the experience section list
  experience_section = driver.find_elements(By.XPATH, '//*[@id="experience"]/parent::section//li[contains(@class,"artdeco-list__item")]')
  experiences = []
  for experience in experience_section:
    # get the content box
    contentbox = experience.find_element(By.XPATH, './/div[@class="display-flex flex-row justify-space-between"]')
    # Get Text from content Box
    texts = contentbox.find_elements(By.XPATH, './/span[@aria-hidden="true"]')
    # Print Each Experience
    experience_texts = []
    for text in texts:
      experience_texts.append(text.text.strip())
    experiences.append(experience_texts)
  return experiences

In [ ]:
def get_education(driver):
  # Click Show all educations.
  show_all_xpath = '//a[@id="navigation-index-see-all-education"]'
  # check if the show all a tag exists
  show_all_link = driver.find_elements(By.XPATH, show_all_xpath)
  show_all = False
  if len(show_all_link) > 0:
    show_all_link[0].click()
    # flag status of show all link
    show_all = True
    time.sleep(WAIT_TIME)
  # find the education section list
  if show_all:
    education_section = driver.find_elements(By.XPATH, '//li[contains(@class,"artdeco-list__item")]')
  else:
    education_section = driver.find_elements(By.XPATH, '//div[@id="education"]/parent::section//li[contains(@class,"artdeco-list__item")]')
  educations = []
  # iterate through each education
  for education in education_section:
    # Get Text from education
    texts = education.find_elements(By.XPATH, './/span[@aria-hidden="true"]')
    education_texts = []
    for text in texts:
      education_texts.append(text.text.strip())
    educations.append(education_texts)
  # navigate back to profile page if show all link exists
  if show_all:
    driver.back()
    time.sleep(WAIT_TIME)
  return educations

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_profile(driver, profile_url):
    driver.get(profile_url)
    time.sleep(3)
    profile = {"name": None, "location": None, "experiences": [], "educations": []}
    try:
        profile["name"] = get_name(driver)
    except Exception as e:
        print(f"Error getting name for {profile_url}: {e}")
    try:
        profile["location"] = get_location(driver)
    except Exception as e:
        print(f"Error getting location for {profile_url}: {e}")
    try:
        profile["experiences"] = get_experience(driver)
    except Exception as e:
        print(f"Error getting experience for {profile_url}: {e}")
    try:
        educations = get_education(driver)
        profile["educations"] = [education[:3] for education in educations] if educations else []
    except Exception as e:
        print(f"Error getting education for {profile_url}: {e}")
    return profile


In [ ]:
def get_driver():
  driver = webdriver.Edge()
  return driver

In [ ]:
all_profiles = []

def main():
  global all_profiles
  driver = get_driver()
  login(driver, USERNAME, PASSWORD)

  for profile_url in profiles:
        try:
            profile = get_profile(driver, profile_url)
            if profile:
                all_profiles.append(profile)
            else:
                print(f"Profile could not be retrieved: {profile_url}")
        except Exception as e:
            print(f"An error occurred with URL {profile_url}: {e}")

In [ ]:
if __name__ == '__main__':
  main()

In [ ]:
len(all_profiles)

159

# Step 2: Compiling scrapped information to Excel file

In [ ]:
df = pd.DataFrame(all_profiles)
anomalous = pd.DataFrame({'anomalous': [None] * len(df)})
df = anomalous.join(df)
df

In [ ]:
df.columns

Index(['anomalous', 'name', 'location', 'experiences', 'educations'], dtype='object')

In [ ]:
print("Columns in DataFrame:", df.columns.tolist())  # List all columns
print("Exact column names:", [repr(col) for col in df.columns])  # Show any invisible characters


Columns in DataFrame: ['anomalous', 'name', 'location', 'experiences', 'educations']
Exact column names: ["'anomalous'", "'name'", "'location'", "'experiences'", "'educations'"]


In [ ]:
df.rename(columns=lambda x: x.strip().lower(), inplace=True)  # Standardize column names


In [ ]:
# Split experiences

max_experiences = df['experiences'].apply(len).max()

for i in range(1, max_experiences+1):
    df[f'experience{i}_role'] = None
    df[f'experience{i}_company'] = None
    df[f'experience{i}_start_date'] = None
    df[f'experience{i}_end_date'] = None

for index, row in df.iterrows():
    for i, experience in enumerate(row['experiences']):
        df.at[index, f'experience{i+1}_role'] = experience[0] if len(experience) > 0 else None
        df.at[index, f'experience{i+1}_company'] = experience[1] if len(experience) > 1 else None

        start_date = None
        end_date = None

        # Split duration into start and end dates if '-' is present
        if len(experience) > 2:
            if '-' in experience[2]:
                parts = experience[2].split('-', 1)
                start_date = parts[0].strip() if parts[0] else None
                end_date = parts[1].strip() if len(parts) > 1 else None
            else:
                df.at[index, 'anomalous'] = 'Yes'
        else:
            df.at[index, 'anomalous'] = 'Yes'

        # Check if end_date contains additional information and clean it up
        if end_date and '·' in end_date:
            end_date = end_date.split('·', 1)[0].strip()

        # Replace 'present' with the latest datetime
        if end_date and end_date.lower() == 'present':
            end_date = datetime.datetime.now().strftime('%b %Y')

        df.at[index, f'experience{i+1}_start_date'] = start_date
        df.at[index, f'experience{i+1}_end_date'] = end_date

df.drop(['experiences'], axis=1, inplace=True)


In [ ]:
# Split educations

max_educations = df['educations'].apply(len).max()

for i in range(1, max_educations+1):
    df[f'educations{i}_university'] = None
    df[f'educations{i}_degree'] = None
    df[f'educations{i}_duration'] = None

for index, row in df.iterrows():
    for i, education in enumerate(row['educations']):
        df.at[index, f'educations{i+1}_university'] = education[0] if len(education) > 0 else None
        df.at[index, f'educations{i+1}_degree'] = education[1] if len(education) > 1 else None
        df.at[index, f'educations{i+1}_duration'] = education[2] if len(education) > 2 else None

df.drop(['educations'], axis=1, inplace=True)

In [ ]:
# Check the final result
df.head(5)

In [ ]:
# Write to excel file
df.to_excel("ScrappedOutput.xlsx", index=False)

# Step3: Determin the Industry and Sector Category

## Train the Industry model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
df_new = pd.read_csv('./All Programs - database.csv')
df_cleaned = df_new[['FirstCompany','FirstPosition','FirstIndusty','FirstSector','WillingtoContact']]
df_cleaned = df_cleaned.dropna(subset=['FirstCompany', 'FirstPosition', 'FirstIndusty','FirstSector'])
df_cleaned = df_cleaned[df_cleaned['WillingtoContact'].isna()]

In [ ]:
df_cleaned['TextFeatures'] = df_cleaned['FirstCompany'] + ' ' + df_cleaned['FirstPosition']
X = df_cleaned['TextFeatures']
y = df_cleaned['FirstIndusty']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

y_pred_series = pd.Series(y_pred, index=X_test.index)

y_pred_corrected = pd.Series(y_pred, index=X_test.index)

accuracy_corrected = accuracy_score(y_test, y_pred_corrected)
print(f'Accuracy after processing: {accuracy_corrected}')

Accuracy after processing: 0.670995670995671


## Make the Industry Prediction

In [ ]:
for i in range(1, max_experiences + 1):
    role_col = f'experience{i}_role'
    company_col = f'experience{i}_company'
    prediction_col = f'experience{i}_industry_prediction'

    # Find the index of the company column to insert the prediction column after it
    company_col_index = df.columns.get_loc(company_col)

    # Combine role and company with whitespace removed; treat both empty as "Unknown"
    combined_features = df.apply(
        lambda row: f"{str(row[role_col]).strip() if pd.notnull(row[role_col]) else ''} "
                    f"{str(row[company_col]).strip() if pd.notnull(row[company_col]) else ''}".strip(),
        axis=1
    )

    # Generate predictions only if role or company has value
    predictions = [
        None if feature == '' else pipeline.predict([feature])[0]
        for feature in combined_features
    ]

    # Drop existing prediction column if it exists
    if prediction_col in df.columns:
        df.drop(columns=[prediction_col], inplace=True)

    # Insert predictions into the dataframe right after the company column
    df.insert(company_col_index + 1, prediction_col, predictions)


In [ ]:
df.to_excel("LinkedinOutput_industrypredictoin.xlsx", index=False)

## Train the Sector model

In [ ]:
df_new = pd.read_csv('./All Programs - database.csv')
df_cleaned = df_new[['FirstCompany','FirstPosition','FirstIndusty','FirstSector','WillingtoContact']]
df_cleaned = df_cleaned.dropna(subset=['FirstCompany', 'FirstPosition', 'FirstIndusty','FirstSector'])
df_cleaned = df_cleaned[df_cleaned['WillingtoContact'].isna()]

In [ ]:
df_cleaned['TextFeatures'] = df_cleaned['FirstCompany'] + ' ' + df_cleaned['FirstPosition']
X = df_cleaned['TextFeatures']
y = df_cleaned['FirstSector']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

y_pred_series = pd.Series(y_pred, index=X_test.index)

y_pred_corrected = pd.Series(y_pred, index=X_test.index)

y_pred_corrected = y_pred_corrected.index.map(
    lambda idx: "Research" if "research" in X_test[idx].lower() or "fellow" in X_test[idx].lower() else (
                "Education" if "tutor" in X_test[idx].lower() or "professor" in X_test[idx].lower() else (
                "Technology" if "software" in X_test[idx].lower() or
                                ("programmer" in X_test[idx].lower() and not any(bio_term in X_test[idx].lower() for bio_term in ["Bioinformatics", "Bioinformatic","Biostatistical "])) else (
                "Medicine" if "doctor" in X_test[idx].lower() else y_pred_corrected[idx]
                )))
)

accuracy_corrected = accuracy_score(y_test, y_pred_corrected)
print(f'Accuracy after processing: {accuracy_corrected}')

Accuracy after processing: 0.7056277056277056


## Make the Sector prediction

In [ ]:
df = pd.read_excel('./LinkedinOutput_industrypredictoin.xlsx')

In [ ]:
for i in range(1, max_experiences + 1):
    role_col = f'experience{i}_role'
    company_col = f'experience{i}_company'
    prediction_col = f'experience{i}_sector_prediction'

    # Find the index of the company column to insert the prediction column after it
    company_col_index = df.columns.get_loc(company_col)

    # Combine role and company columns with proper handling of missing values
    combined_features = df.apply(
        lambda row: f"{str(row[role_col]).strip() if pd.notnull(row[role_col]) else ''} "
                    f"{str(row[company_col]).strip() if pd.notnull(row[company_col]) else ''}".strip(),
        axis=1
    )

    # Generate predictions: leave blank (None) if both role and company are empty
    predictions = [
        None if feature == '' else pipeline.predict([feature])[0]
        for feature in combined_features
    ]

    # Drop existing prediction column if it exists
    if prediction_col in df.columns:
        df.drop(columns=[prediction_col], inplace=True)

    # Insert predictions into the dataframe right after the company column (+2 offset for safer placement)
    df.insert(company_col_index + 2, prediction_col, predictions)


In [ ]:
df.to_excel("LinkedinOutput_Finalized.xlsx", index=False)